# Towardsdatascience Scraping Tutorial

## 1. The Scraping Script

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Import the needed packages
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import warnings
warnings.filterwarnings('ignore')

#Create dictionaries with needed dataframes
dct_bl = {}
dct_pl = {}
dct_seriea = {}
dct_laliga = {}
dct_ligue1 = {}
dct_russia = {}
dct_portugal = {}
dct_ukraine = {}
dct_turkey = {}
dct_belgium = {}

for m in range(2012,2019):
    dct_bl['df_bl_%s' % m] = pd.DataFrame()
    dct_pl['df_pl_%s' % m] = pd.DataFrame()
    dct_seriea['df_seriea_%s' % m] = pd.DataFrame()
    dct_laliga['df_laliga_%s' % m] = pd.DataFrame()
    dct_ligue1['df_ligue1_%s' % m] = pd.DataFrame()
    dct_russia['df_russia_%s' % m] = pd.DataFrame()
    dct_portugal['df_portugal_%s' % m] = pd.DataFrame()
    dct_ukraine['df_ukraine_%s' % m] = pd.DataFrame()
    dct_turkey['df_turkey_%s' % m] = pd.DataFrame()
    dct_belgium['df_belgium_%s' % m] = pd.DataFrame()
    
#List with our needed URLs for each league
league_urls = (['https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=',
               'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=',
               'https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=',
               'https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=',
               'https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=',
               'https://www.transfermarkt.com/premier-liga/startseite/wettbewerb/RU1/plus/?saison_id=',
                'https://www.transfermarkt.com/liga-nos/startseite/wettbewerb/PO1/plus/?saison_id=',
                'https://www.transfermarkt.com/premier-liga/startseite/wettbewerb/UKR1/plus/?saison_id=',
                'https://www.transfermarkt.com/super-lig/startseite/wettbewerb/TR1/plus/?saison_id=',
                'https://www.transfermarkt.com/jupiler-pro-league/startseite/wettbewerb/BE1/plus/?saison_id='])

#Scrapping part
#The first loop is for each url in our URL-list
for l in range(0, len(league_urls)):
    time.sleep(2)
    #The second loop is for each year we want to scrape
    for n in range(2012,2019):
        time.sleep(3)
        df_soccer1 = None
        url = league_urls[l] + str(n)
        headers = {"User-Agent":"Mozilla/5.0"}
        response = requests.get(url, headers=headers, verify=False)
        time.sleep(3)
        soup = BeautifulSoup(response.text, 'html.parser')

        #Table 1 with information about the value
        table = soup.find("table", {"class" : "items"})

        team = []
        squad = []
        age = []
        foreigners = []
        total_market_value = []
        average_market_value = []

        for row in table.findAll('tr'):
            try:
                col = row.findAll('td')
                team.append(col[2].text)
                squad.append(col[3].text)
                age.append(col[4].text)
                foreigners.append(col[5].text)
                total_market_value.append(col[6].text)
                average_market_value.append(col[7].text)
            except:
                pass

        team = [elem.replace('\n','').replace('\xa0','').strip() for elem in team]

        #Table 2 with information about placement, goals and points
        df_soccer2 = None

        table2 = soup.findAll("div", {"class" : "responsive-table"})

        team2 = []
        place = []
        matches = []
        difference = []
        pts = []

        if len(table2) <= 2:
            for row in table2[1].findAll('tr'):
                try:
                    col = row.findAll('td')
                    team2.append(col[2].text)
                    place.append(col[0].text)
                    matches.append(col[3].text)
                    difference.append(col[4].text)
                    pts.append(col[5].text)
                except:
                    pass
        else:
            #Sometimes the information you need is in another table
            for row in table2[2].findAll('tr'):
                try:
                    col = row.findAll('td')
                    team2.append(col[2].text)
                    place.append(col[0].text)
                    matches.append(col[3].text)
                    difference.append(col[4].text)
                    pts.append(col[5].text)
                except:
                    pass

        team2 = [elem.replace('\n','').replace('\xa0','').strip() for elem in team2]

        df_soccer1 = pd.DataFrame({'Team': team[1:], 'Season': n, 'Squad': squad[1:], 'Age': age[1:], 'Foreigners': foreigners[1:],
                                     'Total Value': total_market_value[1:], 'Average value': average_market_value[1:]})

        df_soccer2 = pd.DataFrame({'Team': team2, 'Place': place, 'Matches': matches, 'Difference': difference,
                                 'Points': pts})
        
        #Store all dictionaries in a list
        dct_all = [dct_bl,dct_pl,dct_seriea,dct_laliga,dct_ligue1,dct_russia,dct_portugal,dct_ukraine,dct_turkey,dct_belgium]
        
        #Merge df_soccer1 and df_soccer2 for each season
        dct_all[l]['df_bl_%s' % n] = pd.merge(df_soccer1, df_soccer2, how="inner", left_on="Team", right_on="Team")

## 2. Data Transformation

In [ ]:
df_bl_final = pd.concat(dct_bl.values(), ignore_index=True)
df_pl_final = pd.concat(dct_pl.values(), ignore_index=True) 
df_seriea_final = pd.concat(dct_seriea.values(), ignore_index=True)
df_laliga_final = pd.concat(dct_laliga.values(), ignore_index=True)
df_ligue1_final = pd.concat(dct_ligue1.values(), ignore_index=True)
df_russia_final = pd.concat(dct_russia.values(), ignore_index=True)
df_portugal_final = pd.concat(dct_portugal.values(), ignore_index=True)
df_ukraine_final = pd.concat(dct_ukraine.values(), ignore_index=True)
df_turkey_final = pd.concat(dct_turkey.values(), ignore_index=True)
df_belgium_final = pd.concat(dct_belgium.values(), ignore_index=True)

df_bl_final['Country'] = 'Germany'
df_pl_final['Country'] = 'England'
df_seriea_final['Country'] = 'Italy'
df_laliga_final['Country'] = 'Spain'
df_ligue1_final['Country'] = 'France'
df_russia_final['Country'] = 'Russia'
df_portugal_final['Country'] = 'Portugal'
df_ukraine_final['Country'] = 'Ukraine'
df_turkey_final['Country'] = 'Turkey'
df_belgium_final['Country'] = 'Belgium'

df_final = pd.concat([df_bl_final,df_pl_final,df_seriea_final,df_laliga_final,df_ligue1_final,
                df_russia_final,df_portugal_final,df_ukraine_final,df_turkey_final,df_belgium_final], ignore_index=True)

In [ ]:
df_final.head(5)

In [ ]:
df_final.loc[(df_final['Team'] == 'Bayern Munich') & (df_final['Season'] == 2018)]
df_final.loc[(df_final['Team'] == 'KV Oostende') & (df_final['Season'] == 2018)]
df_final.loc[(df_final['Team'] == 'Real Madrid') & (df_final['Season'] == 2018)]

In [ ]:
#Cleaning strings and remove letters
list_dataframes_conv = (['Average value','Total Value'])

for n in range(0,len(list_dataframes_conv)):
    df_final[list_dataframes_conv[n]] = df_final[list_dataframes_conv[n]]. \
    apply(lambda x: (x + "0").replace(',','') if " Bill. €" in x else x).str.replace(' Bill. €','')

    df_final[list_dataframes_conv[n]] = df_final[list_dataframes_conv[n]]. \
    apply(lambda x: "0," + x if " Th. €" in x else x).str.replace(' Mill. €','').str.replace(' Th. €','').replace(',',np.NaN)

    df_final[list_dataframes_conv[n]] = df_final[list_dataframes_conv[n]].apply(lambda x: x.replace(',','.'))

df_final['Age'] = df_final['Age'].apply(lambda x: x.replace(',','.'))

In [ ]:
#Convert columns for calculation to float
df_final[['Squad', 'Age', 'Foreigners', 'Total Value','Average value', 'Place', 'Matches', 'Difference', 'Points']] = \
df_final[['Squad', 'Age', 'Foreigners', 'Total Value','Average value', 'Place', 'Matches', 'Difference', 'Points']].astype(float)

In [ ]:
#df_final.to_pickle("...path\\df_final.pkl")
#df_final.to_excel("...path\\df_final.xlsx")

## 3. Analysis

In [ ]:
#Plotting the development
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
cufflinks.set_config_file(world_readable=True, theme='pearl')

#Development of the Squad values
mepp_list_avg = []

for n in range(2012,2019):
    mepp = df_final[df_final['Season'] == n]['Total Value'].mean()
    mepp_list_avg.append(mepp)

#List with our Years
years_l = []
for k in range(2012,2019,1):
    years_l.append(k)

trace0 = go.Scatter(
    x = years_l,
    y = mepp_list_avg,
    name = 'Average'
)

layout = {
  "title": "Development of Million € Squad Value per Point", 
  "font": {"size": 16}, 
  "yaxis": {
    "title": "Million € Squad Value per Point", 
  }
}

data = ([trace0])

fig= go.Figure(data=data, layout=layout)
plotly.offline.plot(fig)

In [ ]:
#Correlation
df_final.corr()['Total Value']['Points']

In [ ]:
#Regression
import statsmodels.api as sm
reg1 = sm.OLS(endog=df_final['Points'], exog=df_final[['Total Value']], missing='drop')
type(reg1)
results = reg1.fit()
type(results)
print(results.summary())